In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

from sklearn.linear_model import LinearRegression

data = pd.read_csv('Coffee-Shop-Sales.csv')
data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='mixed')
data['revenue'] = data['transaction_qty'] * data['unit_price']



In [2]:
# Ambil data pendapatan per bulan untuk setiap wilayah
pendapatan_perbulan_perwilayah = data.groupby([data['store_location'], data['transaction_date'].dt.to_period('M')])['revenue'].sum()

# Pisahkan data pendapatan per bulan untuk setiap wilayah
pendapatan_astoria = pendapatan_perbulan_perwilayah.loc['Astoria']
pendapatan_HK = pendapatan_perbulan_perwilayah.loc["Hell's Kitchen"]
pendapatan_LM = pendapatan_perbulan_perwilayah.loc['Lower Manhattan']

# Dapatkan data bulan dari Juli sampai Desember
bulan_prediksi = np.arange(len(pendapatan_astoria), len(pendapatan_astoria) + 6).reshape(-1, 1)

# Latih model regresi linear untuk setiap wilayah
model_astoria = LinearRegression().fit(np.arange(len(pendapatan_astoria)).reshape(-1, 1), pendapatan_astoria)
model_HK = LinearRegression().fit(np.arange(len(pendapatan_HK)).reshape(-1, 1), pendapatan_HK)
model_LM = LinearRegression().fit(np.arange(len(pendapatan_LM)).reshape(-1, 1), pendapatan_LM)

# Prediksi pendapatan untuk bulan Juli sampai Desember
prediksi_astoria = model_astoria.predict(bulan_prediksi)
prediksi_HK = model_HK.predict(bulan_prediksi)
prediksi_LM = model_LM.predict(bulan_prediksi)

# Visualisasikan
fig = go.Figure()

# Tambahkan plot sample untuk bulan Januari sampai Juni
fig.add_trace(go.Scatter(x=pendapatan_astoria.index.astype(str), y=pendapatan_astoria, mode='lines', name='Astoria (Sample)'))
fig.add_trace(go.Scatter(x=pendapatan_HK.index.astype(str), y=pendapatan_HK, mode='lines', name="Hell's Kitchen (Sample)"))
fig.add_trace(go.Scatter(x=pendapatan_LM.index.astype(str), y=pendapatan_LM, mode='lines', name='Lower Manhattan (Sample)'))

# Tambahkan plot prediksi untuk bulan Juli sampai Desember
fig.add_trace(go.Scatter(x=pendapatan_astoria.index[-1:].astype(str).append(pd.period_range(start='2023-07', periods=6, freq='M')).astype(str), y=np.concatenate((pendapatan_astoria[-1:], prediksi_astoria)), mode='lines', name='Astoria (Prediksi)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=pendapatan_HK.index[-1:].astype(str).append(pd.period_range(start='2023-07', periods=6, freq='M')).astype(str), y=np.concatenate((pendapatan_HK[-1:], prediksi_HK)), mode='lines', name="Hell's Kitchen (Prediksi)", line=dict(color='orange')))
fig.add_trace(go.Scatter(x=pendapatan_LM.index[-1:].astype(str).append(pd.period_range(start='2023-07', periods=6, freq='M')).astype(str), y=np.concatenate((pendapatan_LM[-1:], prediksi_LM)), mode='lines', name='Lower Manhattan (Prediksi)', line=dict(color='green')))

# Menambahkan layout
fig.update_layout(title='Prediksi total pendapatan bulanan per lokasi',
                  xaxis_title='Bulan',
                  yaxis_title='Pendapatan')

# Menampilkan plot
fig.show()